In [45]:
query = "grading policy | marks distribution"

In [2]:
from langchain_community.document_loaders import JSONLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma

In [3]:
import json
from pathlib import Path
from pprint import pprint

In [4]:
file_path='../data/test.json'

In [12]:
json_data = json.loads(Path(file_path).read_text())

In [13]:
print(len(json_data['docs']))

10


In [14]:
from gpt4all import Embed4All
import numpy as np

In [46]:
# Initialize Embed4All
embeddings = Embed4All()
query_vec = embeddings.embed(query)

In [19]:
secondary_data = json.load(open('../data/secondary_data.json'))
print(secondary_data)


{'Judson-MATH-1A-01-W19': {'grading': 'General Grading: Each lab member is required to turn in their own lab report. Failure to turn in a lab\nreport will result in a 0 for that lab member. There will be no late labs accepted. Each lab will be graded\nout of 100 points. Except where indicated on an individual lab, I will randomly select different lab reports\nto assess for each section of the lab. Every member of the lab group will receive the same score for a. If you are taking the honors section of this course you will be required to do the honors problems\non the homework assignments. These problems will represent half of your homework grade. In\nlieu of your discussion grade you will also complete an honors project. The honors project will be\na somewhat shorter version of a lab assignment that you will complete individually.\n\nGrading Scale\n\nDue to the complexity of the material the grading scale we will use is as follows. the homework must be stapled together. Four points of y

In [53]:
secondary_data = {}

In [54]:
from gpt4all import Embed4All
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Initialize Embed4All
embeddings = Embed4All()

# # Define the query string
# query = "your query string"

# # Embed the query
# query_vec = embeddings.embed(query)

# Iterate over each document
for doc_id in range(len(json_data['docs'])):
    doc_str = json_data['docs'][doc_id]['content']

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
    doc_split = text_splitter.create_documents([doc_str])

    # Embed the documents
    doc_vecs = np.array([embeddings.embed(sub_doc.page_content) for sub_doc in doc_split])

    # Compute cosine similarity scores
    scores = cosine_similarity([query_vec], doc_vecs)[0]

    # Get top k documents
    k = 5
    top_docs_indices = np.argsort(-scores)[:k]
    top_docs = [doc_split[i] for i in top_docs_indices]

    # print("title:", json_data['docs'][doc_id]['title'])
    # print("matching substrings:")
    final_doc_str = ""
    for sub_doc in top_docs:
        # print(sub_doc.page_content)
        final_doc_str += sub_doc.page_content + ". "
    
    if json_data['docs'][doc_id]['title'] not in secondary_data:
        secondary_data[json_data['docs'][doc_id]['title']] = {}
    secondary_data[json_data['docs'][doc_id]['title']][query] = final_doc_str    
    # secondary_data[json_data['docs'][doc_id]['title']]['retrieved_content'] = final_doc_str

    # print("\n")

In [55]:
secondary_data

{'Judson-MATH-1A-01-W19': {'grading policy | marks distribution': 'Grade\nYour grade will be computed using the following grade distribution.\n5% Labs 10% Quizzes. 8 Ch. 4.4\n\nQuiz 9\nHw 13 due\n22 Lab 6due\n\n \n\x0cLab Grading Policies. will be graded out of 5 points each. You will not know which 4 problems will be graded. For the. Grading Scale\n\nDue to the complexity of the material the grading scale we will use is as follows. General Grading: Each lab member is required to turn in their own lab report. Failure to turn in a. '},
 'Lien-MATH-1A-05-W19': {'grading policy | marks distribution': 'Final grades are non-negotiable. You should monitor your scores in the gradebook regularly. QUESTION: What is my current grade in the class?. Breakdown of grades:\nHomework\n63-879% «BO\n\n80-82.9% «|B\n63-679% «(DS\n60-62.9% «(| D-. marks incorrectly. Problems must be brought to my attention immediately after exams are returned.. - You are required to come to class prepared WITH lecture not

In [56]:
with open('../data/secondary_data.json', 'w') as outfile:
    json.dump(secondary_data, outfile, indent=4)

## Generation

In [24]:
from langchain_community.llms import Ollama

In [25]:
llm = Ollama(model="phi3")

In [27]:
# llm.invoke("Tell me a joke")

In [57]:
retrieved_data = ""
secondary_data = json.load(open('../data/secondary_data.json'))
print(secondary_data)


{'Judson-MATH-1A-01-W19': {'grading policy | marks distribution': 'Grade\nYour grade will be computed using the following grade distribution.\n5% Labs 10% Quizzes. 8 Ch. 4.4\n\nQuiz 9\nHw 13 due\n22 Lab 6due\n\n \n\x0cLab Grading Policies. will be graded out of 5 points each. You will not know which 4 problems will be graded. For the. Grading Scale\n\nDue to the complexity of the material the grading scale we will use is as follows. General Grading: Each lab member is required to turn in their own lab report. Failure to turn in a. '}, 'Lien-MATH-1A-05-W19': {'grading policy | marks distribution': 'Final grades are non-negotiable. You should monitor your scores in the gradebook regularly. QUESTION: What is my current grade in the class?. Breakdown of grades:\nHomework\n63-879% «BO\n\n80-82.9% «|B\n63-679% «(DS\n60-62.9% «(| D-. marks incorrectly. Problems must be brought to my attention immediately after exams are returned.. - You are required to come to class prepared WITH lecture note

In [58]:
for course in secondary_data:
    print(course)
    retrieved_data += course + " :\n "+ secondary_data[course]["grading policy | marks distribution"] + "\n\n"

Judson-MATH-1A-01-W19
Lien-MATH-1A-05-W19
Howard-MATH-1A-07-W19
Mailhot-MATH-1A-09-W19
Nay-MATH-1A-11-W19
Mailhot-MATH-1A-23-W19
Lopez-MATH-1A-25-W19
Howard-MATH-1A-27-W19
Du-MATH-1A-63-W19
Dhaliwal-MATH-1A-67Z-W19


In [59]:
print(retrieved_data)

Judson-MATH-1A-01-W19 :
 Grade
Your grade will be computed using the following grade distribution.
5% Labs 10% Quizzes. 8 Ch. 4.4

Quiz 9
Hw 13 due
22 Lab 6due

 
Lab Grading Policies. will be graded out of 5 points each. You will not know which 4 problems will be graded. For the. Grading Scale

Due to the complexity of the material the grading scale we will use is as follows. General Grading: Each lab member is required to turn in their own lab report. Failure to turn in a. 

Lien-MATH-1A-05-W19 :
 Final grades are non-negotiable. You should monitor your scores in the gradebook regularly. QUESTION: What is my current grade in the class?. Breakdown of grades:
Homework
63-879% «BO

80-82.9% «|B
63-679% «(DS
60-62.9% «(| D-. marks incorrectly. Problems must be brought to my attention immediately after exams are returned.. - You are required to come to class prepared WITH lecture notes printed out.

GRADING. 

Howard-MATH-1A-07-W19 :
 Grading:. the course, and letter grades will be deter

In [23]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [68]:
prompt = "Analyse the grading policies from the following retrieved docs of various courses. Summarize the information under 1 sentence. Be very concise with your answer and to the point." + retrieved_data

In [69]:
llm.invoke(prompt)

"Judson-MATH-1A-01-W19 grading includes lab work (5%), quizzes (10%), and chapter 4 assignments (8%). Howard-MATH-1A-27-W19 employs a curve set at the instructor's discretion, with potential adjustments after key exams. Lopez-MATH-1A-25-W19 has strict penalties for incorrect answers or changes in grades post assessment submissions. Dhaliwal-MATH-1A-67Z-W19 utilizes a weighted grading system of 15% quizzes, 45% exams (3), and 20% final exam with no grade drops.<|end|><|endoftext|>"